<a href="https://colab.research.google.com/github/sriharikrishna/EuroAD26/blob/main/diffrax_odetest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
!pip uninstall diffrax
!pip install diffrax
#!pip install git+https://github.com/patrick-kidger/diffrax.git


Found existing installation: diffrax 0.4.1
Uninstalling diffrax-0.4.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/diffrax-0.4.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/diffrax/*
    /usr/local/lib/python3.10/dist-packages/test/*
Proceed (Y/n)? y
  Successfully uninstalled diffrax-0.4.1
  Using cached diffrax-0.4.1-py3-none-any.whl (161 kB)


In [57]:
from diffrax import diffeqsolve, ODETerm, Dopri5, Tsit5
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

In [58]:
def solver(y0, t_i, t_f, A):
  def ode_fn(t, y, B):
    return jnp.matmul(B[0],y)
  term = ODETerm(ode_fn)
  #ODEsolver = Dopri5()
  ODEsolver = Tsit5()
  solution = diffeqsolve(term, ODEsolver, t0=t_i, t1=t_f, dt0=0.2, y0=y0, args=(A,))
  return solution.ys


In [59]:
t_i = 0.
t_f = jnp.array(2., dtype=jnp.float64)

y0 = jnp.array([[1.0, 9.0],
               [1.0, 9.0]], dtype=jnp.float64)
A = jnp.array([[0, 1.0],
               [- 100.0, 0]], dtype=jnp.float64)

solution = solver(y0, t_i, t_f, A)

print(solution)

[[[  0.66391629   5.97524664]
  [-11.09835404 -99.8851864 ]]]


In [60]:
t_i = 0.
t_f = jnp.array(2., dtype=jnp.float64)

y0 = jnp.array([[1.0-1j, 9.0-2j],
                [1.0-1j, 9.0-2j]], dtype=jnp.complex128)
A = jnp.array([[0-1j, 1.0+2j],
               [- 100.0+3j, 0+4j]], dtype=jnp.complex128)

solution = solver(y0, t_i, t_f, A)

print(solution)

[[[ 1724938.16948032 +1617123.08186009j
    3827229.14563176+14931460.54341212j]
  [ 3648413.14946532-13206359.14799311j
   66288529.34003741-59865529.29083761j]]]


In [61]:
t_i = jnp.array(0.+0j, dtype=jnp.complex128)
t_f = jnp.array(2.+0j, dtype=jnp.complex128)
y0 = jnp.array([[1.0-1j, 9.0-2j],
                [1.0-1j, 9.0-2j]], dtype=jnp.complex128)

A = jnp.array([[0-1j, 1.0+2j],
               [- 100.0+3j, 0+4j]], dtype=jnp.complex128)
jacrev_fun = jax.jacrev(solver, argnums=(2),holomorphic=True)
jac = jacrev_fun(y0, t_i, t_f, A)
print(jac)

[[[ 3.49859488e+07-1.05739384e+07j  2.29431503e+08+6.42941598e+07j]
  [-1.48511892e+08-1.50992711e+08j -2.88340914e+08-1.35025153e+09j]]]
